In [7]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Flatten, Dense, concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import CategoricalAccuracy, MeanIoU
import matplotlib.pyplot as plt

# Set your paths
train_path = '/kaggle/input/cs770-final-project-skin-lesion-segmentation/Project_Data/Train'

# Load ground truth information
ground_truth = pd.read_csv(os.path.join(train_path, 'GroundTruth.csv'))

# Load Images, Masks, and Labels
images = [img_to_array(load_img(os.path.join(train_path, 'images', img + '.jpg'), target_size=(128, 128))) for img in ground_truth['image']]
masks = [img_to_array(load_img(os.path.join(train_path, 'masks', img + '_segmentation.png'), target_size=(128, 128), color_mode='grayscale')) for img in ground_truth['image']]
labels = ground_truth[['MEL', 'NV', 'BCC', 'AKIEC', 'BKL', 'DF', 'VASC']].values

# Convert lists to numpy arrays
images = np.array(images) / 255.0  # Normalize pixel values to [0, 1]
masks = np.array(masks) / 255.0  # Normalize pixel values to [0, 1]

# Split data into training and validation sets
images_train, images_val, masks_train, masks_val, labels_train, labels_val = train_test_split(
    images, masks, labels, test_size=0.2, random_state=42
)

# Model for classification
input_shape = (128, 128, 3)  # Update this based on your image size
num_classes_classification = 7  # Multi-class classification

# Classification input
inputs_classification = Input(shape=input_shape, name='classification_input')

# Encoder
conv1_classification = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs_classification)
pool1_classification = MaxPooling2D(pool_size=(2, 2))(conv1_classification)

# Classification branch
flat_classification = Flatten()(pool1_classification)
dense1_classification = Dense(128, activation='relu')(flat_classification)
outputs_classification = Dense(num_classes_classification, activation='softmax', name='classification')(dense1_classification)

# Model for segmentation
num_classes_segmentation = 1  # Binary segmentation

# Segmentation input
inputs_segmentation = Input(shape=input_shape, name='segmentation_input')

# Encoder
conv1_segmentation = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs_segmentation)
pool1_segmentation = MaxPooling2D(pool_size=(2, 2))(conv1_segmentation)

# Decoder
up1_segmentation = UpSampling2D(size=(2, 2))(pool1_segmentation)
decoded_segmentation = Conv2D(num_classes_segmentation, (3, 3), activation='sigmoid', padding='same', name='segmentation')(up1_segmentation)

# Combine both branches
combined_model = Model(inputs=[inputs_classification, inputs_segmentation], outputs=[outputs_classification, decoded_segmentation])

# Use appropriate loss functions and metrics for each branch
combined_model.compile(optimizer=Adam(),
                       loss={'classification': 'categorical_crossentropy', 'segmentation': 'binary_crossentropy'},
                       metrics={'classification': CategoricalAccuracy(), 'segmentation': MeanIoU(num_classes=2)})

# Train the combined model
combined_model.fit(
    {'classification_input': images_train, 'segmentation_input': images_train},
    {'classification': labels_train, 'segmentation': masks_train},
    validation_data=(
        {'classification_input': images_val, 'segmentation_input': images_val},
        {'classification': labels_val, 'segmentation': masks_val}
    ),
    epochs=10, batch_size=32
)

Epoch 1/10
251/251 [==============================] - 265s 1s/step - loss: 2.0711 - classification_loss: 1.5970 - segmentation_loss: 0.4741 - classification_categorical_accuracy: 0.6595 - segmentation_mean_io_u_2: 0.3676 - val_loss: 1.3307 - val_classification_loss: 0.9215 - val_segmentation_loss: 0.4092 - val_classification_categorical_accuracy: 0.6710 - val_segmentation_mean_io_u_2: 0.3639
Epoch 2/10
251/251 [==============================] - 261s 1s/step - loss: 1.2282 - classification_loss: 0.8308 - segmentation_loss: 0.3974 - classification_categorical_accuracy: 0.6936 - segmentation_mean_io_u_2: 0.3676 - val_loss: 1.2339 - val_classification_loss: 0.8348 - val_segmentation_loss: 0.3991 - val_classification_categorical_accuracy: 0.6940 - val_segmentation_mean_io_u_2: 0.3639
Epoch 3/10
251/251 [==============================] - 290s 1s/step - loss: 1.1269 - classification_loss: 0.7448 - segmentation_loss: 0.3821 - classification_categorical_accuracy: 0.7287 - segmentation_mean_io_u

In [8]:
import cv2

# Set your paths for testing data
test_path = '/kaggle/input/cs770-final-project-skin-lesion-segmentation/Project_Data/Test'

# Load ground truth information for testing
ground_truth_test = pd.read_csv('/kaggle/input/cs770-final-project-skin-lesion-segmentation/Project_Data/Test/ISIC2018_Task3_Test_GroundTruth/ISIC2018_Task3_Test_GroundTruth.csv')

# Load Classification Test Images and Labels
images_test_classification = [img_to_array(load_img(os.path.join(test_path, 'ISIC2018_Task3_Test_Input', img + '.jpg'), target_size=(128, 128))) for img in ground_truth_test['image']]
labels_test_classification = ground_truth_test[['MEL', 'NV', 'BCC', 'AKIEC', 'BKL', 'DF', 'VASC']].values

# Convert to numpy arrays and normalize
images_test_classification = np.array(images_test_classification) / 255.0

# Directory path for segmentation images and masks
test_path_segmentation_images = '/kaggle/input/cs770-final-project-skin-lesion-segmentation/Project_Data/Test/ISBI2016_ISIC_Part1_Test_Data'
test_path_segmentation_masks = '/kaggle/input/cs770-final-project-skin-lesion-segmentation/Project_Data/Test/ISBI2016_ISIC_Part1_Test_GroundTruth'

# Get all .jpg files in the segmentation image and masks directory
segmentation_image = [f for f in os.listdir(test_path_segmentation_images) if f.endswith('.jpg')]
segmentation_mask = [f for f in os.listdir(test_path_segmentation_masks) if f.endswith('.png')]

# Sort the lists of image and mask filenames
segmentation_image.sort()
segmentation_mask.sort()

# Load the segmentation images and masks
images_test_segmentation = [img_to_array(load_img(os.path.join(test_path_segmentation_images, img), target_size=(128, 128))) for img in segmentation_image]
masks_test_segmentation_RGB = [img_to_array(load_img(os.path.join(test_path_segmentation_masks, img), target_size=(128, 128))) for img in segmentation_mask]
masks_test_segmentation = [cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)[:, :, np.newaxis] for mask in masks_test_segmentation_RGB]

# Take the first 379 images
images_test_classification = images_test_classification[:379]

# Convert to numpy arrays and normalize
images_test_segmentation = np.array(images_test_segmentation) / 255.0
masks_test_segmentation = np.array(masks_test_segmentation) / 255.0

# Make predictions on the classification test data
classification_predictions,segmentation_predictions = combined_model.predict(
    [images_test_classification, images_test_segmentation]
)



12/12 [==============================] - 3s 229ms/step


In [9]:
# Evaluate the model on the validation set
val_metrics = combined_model.evaluate(
    {'classification_input': images_val, 'segmentation_input': images_val},
    {'classification': labels_val, 'segmentation': masks_val},
    batch_size=32
)

# Print the final metrics
print("Final Classification Accuracy:", val_metrics[3])  # Adjust the index if needed
print("Final Segmentation Mean IoU:", val_metrics[4])  # Adjust the index if needed


63/63 [==============================] - 16s 259ms/step - loss: 1.2417 - classification_loss: 0.8769 - segmentation_loss: 0.3649 - classification_categorical_accuracy: 0.7179 - segmentation_mean_io_u_2: 0.3639
Final Classification Accuracy: 0.717923104763031
Final Segmentation Mean IoU: 0.3638749420642853


In [10]:
combined_model.summary()


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 classification_input (Inpu  [(None, 128, 128, 3)]        0         []                            
 tLayer)                                                                                          
                                                                                                  
 conv2d_2 (Conv2D)           (None, 128, 128, 32)         896       ['classification_input[0][0]']
                                                                                                  
 segmentation_input (InputL  [(None, 128, 128, 3)]        0         []                            
 ayer)                                                                                            
                                                                                            